In [1]:
!pip install simpletransformers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from typing import Dict, List, Any
import logging
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
import jsonlines
from simpletransformers.classification import ClassificationModel, ClassificationArgs

import random 
import numpy as np
import torch

def random_seed(seed_value): 
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value)
    random.seed(seed_value) 


random_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 957.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 15.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 31.4 MB/s eta 0:00:0000:01:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=4fc45b98262e7249849fad504cfacfcf6fff0abdf939b6798a16be372d262b6d
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=3dfd62e02ca2f5511b9dce5bb9c0d7606efed787f1d49de2d98084d169776220
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9

In [2]:
def read_give_instances(path):
    
    texts = []
    labels = []
    with jsonlines.open(path) as f_in:
        for json_object in f_in:
            text=json_object.get('text')
            label=json_object.get('label')
            metadata=json_object.get('metadata')
            texts.append(text)
            labels.append(label)
            
    return texts, labels

In [3]:
train_texts, train_labels = read_give_instances("../input/scibert-dataset/text_classification/sci-cite/train.txt")
dev_texts, dev_labels = read_give_instances("../input/scibert-dataset/text_classification/sci-cite/dev.txt")
test_texts, test_labels = read_give_instances("../input/scibert-dataset/text_classification/sci-cite/test.txt")

In [4]:
np.unique(train_labels)

array(['background', 'method', 'result'], dtype='<U10')

In [5]:
train_texts[4], train_labels[:5]

('Ophthalmic symptoms are rare manifestations of the intracranial arachnoid cyst, and include unilateral exophthalmos, visual field abnormality, decreased visual acuity and isolated palsies of the third, fourth and sixth cranial nerves [1–5].',
 ['background', 'background', 'background', 'background', 'background'])

In [6]:
df_train = pd.DataFrame({"text":train_texts,"labels":train_labels})
df_val = pd.DataFrame({"text":dev_texts,"labels":dev_labels})
df_test = pd.DataFrame({"text":test_texts,"labels":test_labels})
df_train.head()

,text,labels
0,"However, how frataxin interacts with the Fe-S ...",background
1,"In the study by Hickey et al. (2012), spikes w...",background
2,"The drug also reduces catecholamine secretion,...",background
3,By clustering with lowly aggressive close kin ...,background
4,Ophthalmic symptoms are rare manifestations of...,background


In [7]:
len(df_train['labels'].unique())

3

In [8]:
lb = LabelEncoder()

df_train['labels'] = lb.fit_transform(df_train['labels'])
df_val['labels'] = lb.transform(df_val['labels'])
df_test['labels'] = lb.transform(df_test['labels'])
df_train.head()

,text,labels
0,"However, how frataxin interacts with the Fe-S ...",0
1,"In the study by Hickey et al. (2012), spikes w...",0
2,"The drug also reduces catecholamine secretion,...",0
3,By clustering with lowly aggressive close kin ...,0
4,Ophthalmic symptoms are rare manifestations of...,0


In [9]:
len(df_train['labels'].unique())

3

In [10]:
model_args = ClassificationArgs(eval_batch_size=64,
                               learning_rate = 2e-5,
                               max_seq_length = 256,
                               no_save = True,
                               num_train_epochs = 4,
                               train_batch_size = 32)

In [11]:
model = ClassificationModel("bert","AnonymousSub/fpdm_models_scibert_hybrid_epochs_4",args = model_args,num_labels=3)

Downloading:   0%|          | 0.00/706 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AnonymousSub/fpdm_models_scibert_hybrid_epochs_4 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/427 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [12]:
model.train_model(df_train,eval_df = df_val)

  0%|          | 0/7320 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/229 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/229 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/229 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/229 [00:00<?, ?it/s]

(916, 0.2772515519770212)

In [13]:
predictions, raw_outputs = model.predict(df_test['text'].tolist())

  0%|          | 0/1861 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [14]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score

In [15]:
print(classification_report(df_test['labels'], predictions))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88       997
           1       0.90      0.82      0.86       605
           2       0.71      0.90      0.79       259

    accuracy                           0.86      1861
   macro avg       0.83      0.87      0.84      1861
weighted avg       0.87      0.86      0.86      1861



In [16]:
print(accuracy_score(df_test['labels'], predictions))
print(precision_score(df_test['labels'], predictions,average='macro'))
print(recall_score(df_test['labels'], predictions,average='macro'))
print(f1_score(df_test['labels'], predictions,average='macro'))

0.8602901665771091
0.8323383081463719
0.8662933758037251
0.8447286136344992
